In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('C:/Users/HP/Desktop/Traineeship/data_OMI/tourist_class_mun.csv', encoding='latin1', usecols=lambda col: not col.startswith('Unnamed'))

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7926 entries, 0 to 7925
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   region    7926 non-null   object
 1   prov      7926 non-null   object
 2   mun_name  7925 non-null   object
 3   cat_code  7926 non-null   object
 4   cat_prev  7926 non-null   object
 5   cat_list  2208 non-null   object
 6   D         6222 non-null   object
 7   P         6222 non-null   object
 8   T         6222 non-null   object
 9   S         6222 non-null   object
dtypes: object(10)
memory usage: 619.3+ KB


In [39]:
columns = ['cat_code', 'cat_prev', 'cat_list']

for col in columns:
    print(f'Unique values in {col}: {df[col].unique()}')

Unique values in cat_code: ['P' 'E' 'H' 'B' 'Q' 'C' 'L1' 'G' 'F' 'L2' 'A' 'D']
Unique values in cat_prev: ['Comuni turistici non appartenenti ad una categoria specifica'
 'Comuni con vocazione montana'
 'Comuni a vocazione montana e con vocazione culturale, storica, artistica e paesaggistica'
 'Comuni a vocazione culturale, storica, artistica e paesaggistica'
 'Comuni non turistici' 'Comuni con vocazione marittima'
 'Comuni a vocazione culturale, storica, artistica e paesaggistica e altre vocazioni'
 'Comuni a vocazione marittima e con vocazione culturale, storica, artistica e paesaggistica'
 'Comuni del turismo termale' 'Altri comuni turistici con due vocazioni'
 'Grandi città (con turismo multidimensionale)'
 'Comuni del turismo lacuale']
Unique values in cat_list: [nan 'montana' 'montana, culturale' 'culturale' 'marittima'
 'culturale, montana, termale' 'marittima, culturale' 'termale'
 'culturale, termale' 'marittima, termale' 'marittima, montana'
 'culturale, marittima, termale' '

In [40]:
# Dropping 'cat_prev' (expressed by 'cat_code')
df1 = df.drop(columns=['cat_prev'])

In [41]:
# Translate 'cat_list'
rename_dict = {
    'culturale': 'cultural',
    'marittima': 'sea',
    'montana': 'mountain',
    'termale': 'thermal',
    'lacuale': 'lake'
}

# Apply the renaming
def translate_multi_label(s):
    if pd.isna(s) or s == "":
        return ""
    return ', '.join(rename_dict.get(x.strip(), x.strip()) for x in s.split(','))

df1['cat_list'] = df1['cat_list'].apply(translate_multi_label)

In [42]:
# Get dummies names from 'cat_list'
df_dummies = df1['cat_list'].str.get_dummies(sep=", ")

# Merge back into original dataframe
df1 = df1.join(df_dummies)

In [43]:
province_map = {
    'AG': 'AGRIGENTO', 'AL': 'ALESSANDRIA', 'AN': 'ANCONA', 'AO': "VALLE D'AOSTA/VALLÉE D'AOSTE",
    'AR': 'AREZZO', 'AP': 'ASCOLI PICENO', 'AT': 'ASTI', 'AV': 'AVELLINO',
    'BA': 'BARI', 'BT': 'BARLETTA-ANDRIA-TRANI', 'BL': 'BELLUNO', 'BN': 'BENEVENTO',
    'BG': 'BERGAMO', 'BI': 'BIELLA', 'BO': 'BOLOGNA', 'BZ': 'BOLZANO/BOZEN',
    'BS': 'BRESCIA', 'BR': 'BRINDISI', 'CA': 'CAGLIARI', 'CB': 'CAMPOBASSO',
    'CE': 'CASERTA', 'CH': 'CHIETI', 'CL': 'CALTANISSETTA', 'CN': 'CUNEO',
    'CO': 'COMO', 'CR': 'CREMONA', 'CS': 'COSENZA', 'CT': 'CATANIA', 'CZ': 'CATANZARO',
    'EN': 'ENNA', 'FC': "FORLI'-CESENA", 'FE': 'FERRARA', 'FI': 'FIRENZE', 'FG': 'FOGGIA',
    'FR': 'FROSINONE', 'GE': 'GENOVA', 'GO': 'GORIZIA', 'GR': 'GROSSETO', 'IM': 'IMPERIA',
    'IS': 'ISERNIA', 'SP': 'LA SPEZIA', 'LT': 'LATINA', 'LE': 'LECCE', 'LC': 'LECCO',
    'LI': 'LIVORNO', 'LO': 'LODI', 'LU': 'LUCCA', 'MC': 'MACERATA', 'MN': 'MANTOVA',
    'MS': 'MASSA-CARRARA', 'MT': 'MATERA', 'ME': 'MESSINA', 'MI': 'MILANO', 'MO': 'MODENA',
    'MB': 'MONZA E BRIANZA', 'NA': 'NAPOLI', 'NO': 'NOVARA', 'NU': 'NUORO', 'OR': 'ORISTANO',
    'PD': 'PADOVA', 'PA': 'PALERMO', 'PR': 'PARMA', 'PV': 'PAVIA', 'PG': 'PERUGIA',
    'PU': 'PESARO E URBINO', 'PE': 'PESCARA', 'PC': 'PIACENZA', 'PI': 'PISA', 'PT': 'PISTOIA',
    'PN': 'PORDENONE', 'PZ': 'POTENZA', 'PO': 'PRATO', 'RG': 'RAGUSA', 'RA': 'RAVENNA',
    'RC': 'REGGIO DI CALABRIA', 'RE': "REGGIO NELL'EMILIA", 'RI': 'RIETI', 'RN': 'RIMINI',
    'RM': 'ROMA', 'RO': 'ROVIGO', 'SA': 'SALERNO', 'SS': 'SASSARI', 'SV': 'SAVONA',
    'SI': 'SIENA', 'SR': 'SIRACUSA', 'SO': 'SONDRIO', 'TA': 'TARANTO', 'TE': 'TERAMO',
    'TR': 'TERNI', 'TO': 'TORINO', 'TP': 'TRAPANI', 'TN': 'TRENTO', 'TV': 'TREVISO',
    'TS': 'TRIESTE', 'UD': 'UDINE', 'VA': 'VARESE', 'VE': 'VENEZIA', 'VB': 'VERBANO-CUSIO-OSSOLA',
    'VC': 'VERCELLI', 'VR': 'VERONA', 'VI': 'VICENZA', 'VV': 'VIBO VALENTIA', 'VT': 'VITERBO',
    'AQ': "L'AQUILA", 'SU': 'SUD SARDEGNA', 'KR': 'CROTONE', 'FM': 'FERMO'
}
names_missing = [name for name in df1['prov'].unique() if name not in province_map.values()]

print("Province names in df1['prov'] but not in mapping:", names_missing)

Province names in df1['prov'] but not in mapping: []


In [44]:
# Reverse mapping: province name -> code
name_to_code = {name: code for code, name in province_map.items()}

# Apply it to your DataFrame column
df1['prov_code'] = df1['prov'].map(name_to_code)

In [45]:
# Encoding 'D','P','T','S'

columns = ['D','P','T','S']

for c in columns:
    df1[c] = pd.to_numeric(df1[c].str[1:], errors='coerce').astype('Int64').fillna(0)

In [46]:
new_order = [
    'region', 'prov_code', 'mun_name', 'cat_code', 'cultural', 'lake', 'mountain', 'sea', 'thermal', 'cat_list', 'D', 'P', 'T', 'S'
    ]

df1 = df1[new_order]

dict = {
    'D': 'infrastructures',
    'P': 'flows',
    'T': 'economy',
    'S': 'summary'
}

df1 = df1.rename(columns = dict)

df1.head()

,region,prov_code,mun_name,cat_code,cultural,lake,mountain,sea,thermal,cat_list,infrastructures,flows,economy,summary
0,ABRUZZO,AQ,Acciano,P,0,0,0,0,0,,5,1,4,2
1,ABRUZZO,AQ,Aielli,E,0,0,1,0,0,mountain,4,4,4,4
2,ABRUZZO,AQ,Alfedena,H,1,0,1,0,0,"mountain, cultural",4,2,4,3
3,ABRUZZO,AQ,Anversa degli Abruzzi,H,1,0,1,0,0,"mountain, cultural",4,2,5,5
4,ABRUZZO,AQ,Ateleta,E,0,0,1,0,0,mountain,2,2,4,2


In [47]:
df1.to_csv('datasets/tourist_data.csv')